In [1]:
import pandas as pd
import ast
from src.QA import QA 
from src.retrieval import Retrieval
import os
from langsmith import traceable
import dspy
import json
from dspy.teleprompt import MIPROv2
from dspy.evaluate import Evaluate
from dotenv import load_dotenv
import openai
import re

In [ ]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_KEY")
# os.getenv("LANGSMITH_TRACING")
# os.getenv("LANGSMITH_ENDPOINT")
# os.getenv("LANGSMITH_API_KEY")
# os.getenv("LANGSMITH_PROJECT")

In [3]:
lm = dspy.LM('openai/gpt-4o')
dspy.configure(lm=lm)

In [4]:
df = pd.read_csv("data/queries.csv")
df["pred"] = df["pred"].apply(ast.literal_eval)

files = {
    "l_l_24": "линал_лекция_24.json",
    "l_s_24": "линал_семинар_24.json",
    "m_l_12": "матан_лекция_12.json",
    "m_s_12": "матан_семинар_12.json",
    "ma_l_6": "матстат_лекция_6.json",
    "ma_s_12": "матстат_семинар_6.json",
    "t_l_6": "теорвер_лекция_6.json",
    "t_s_6": "теорвер_семинар_6.json",
    "a_l_3": "алгебра_лекция_3.json",
    "a_s_3": "алгебра_семинар_3.json",
}

data = {}

for key, filename in files.items():
    path = f"data/transcriptions/{filename}"
    with open(path, "r", encoding="utf-8") as f:
        data[files[key]] = json.load(f)

retrievers_by_lection = {}

for lection in df["lection"].unique():
    retrieval = Retrieval(k=15)
    texts = [item["text"] for item in data[f"{lection}.json"]]
    metadatas = [{"time": item["time"]} for item in data[f"{lection}.json"]]
    retrieval.add_texts(texts=texts, metadata=metadatas)
    retrievers_by_lection[lection] = retrieval

In [ ]:
class QAWrapper(dspy.Module):
    def __init__(self, pipeline, retrievers_by_lection):
        super().__init__()
        self.pipeline = pipeline
        self.retrievers = retrievers_by_lection

    def forward(self, query, lection=None):
        retriever = self.retrievers[lection]
        preds = self.pipeline.forward_train(query, retriever)
        return dspy.Prediction(ranked_context=preds[:3])

def time_to_seconds(time_str):
    match = re.search(r"(\d+):(\d+)", time_str)
    if not match:
        return None
    minutes, seconds = map(int, match.groups())
    return minutes * 60 + seconds

def accuracy_at_3_metric(example, pred, signature=None):
    true_time = time_to_seconds(example.ranked_context)
    if true_time is None:
        return 0
    
    for candidate in pred.ranked_context:
        pred_time = time_to_seconds(candidate)
        if pred_time is not None and abs(pred_time - true_time) <= 120:
            return 1

    return 0

def accuracy_at_1_metric(example, pred, signature=None):
    true_time = time_to_seconds(example.ranked_context)
    if true_time is None or not pred.ranked_context:
        return 0
    
    first_pred = pred.ranked_context[0]
    pred_time = time_to_seconds(first_pred)

    if pred_time is not None and abs(pred_time - true_time) <= 120:
        return 1
    return 0

# def accuracy_at_3_metric(example, pred, signature=None):
#     return int(example.ranked_context in pred.ranked_context)

# def accuracy_at_1_metric(example, pred, signature=None):
#     if isinstance(pred.ranked_context, str):
#         first = pred.ranked_context
#     elif isinstance(pred.ranked_context, list) and len(pred.ranked_context) > 0:
#         first = pred.ranked_context[0]
#     else:
#         return 0  # ничего нет

#     return int(example.ranked_context == first)


In [6]:
# нормальное разделение на train и valid

trainset = []
valset = []

for lection, group in df.groupby("lection"):
    examples = [
        dspy.Example(
            # context=row["context"],
            query=row["query"],
            ranked_context=row["timestamp"],
            lection=row["lection"]
        ).with_inputs("context", "query", "lection")
        for _, row in group.iterrows()
    ]

    split = int(0.6 * len(examples))
    trainset.extend(examples[:split])
    valset.extend(examples[split:])


# initialize program
program = QAWrapper(pipeline=QA(), retrievers_by_lection=retrievers_by_lection)

evaluate_3 = Evaluate(devset=valset, metric=accuracy_at_3_metric, num_threads=8, display_progress=True, display_table=True)
print(f"Evaluate common program with accuracy@3...")
evaluate_3(program, devset=valset)

evaluate_1 = Evaluate(devset=valset, metric=accuracy_at_1_metric, num_threads=8, display_progress=True, display_table=True)
print(f"Evaluate common program with accuracy@1...")
evaluate_1(program, devset=valset)

Evaluate common program with accuracy@3...
Average Metric: 31.00 / 42 (73.8%): 100%|██████████| 42/42 [00:06<00:00,  6.04it/s]

2025/05/06 15:19:41 INFO dspy.evaluate.evaluate: Average Metric: 31 / 42 (73.8%)


,query,example_ranked_context,lection,pred_ranked_context,accuracy_at_3_metric
0,Определение произведения групп,Время: 44:05,алгебра_лекция_3,[],
1,Определение конечной абелевой группы,Время: 49:05,алгебра_лекция_3,[],
2,Теорема о представлении абелевых групп через произведение циклических,Время: 50:45,алгебра_лекция_3,"[Время: 49:55, Время: 55:45, Время: 76:35]",✔️ [1]
3,Китайская теорема об остатках,Время: 57:25,алгебра_лекция_3,"[Время: 59:55, Время: 57:25, Время: 74:05]",✔️ [1]
4,Каноническое представление абелевых групп,Время: 74:55,алгебра_лекция_3,"[Время: 49:55, Время: 75:45, Время: 76:35]",✔️ [1]
5,Упражнение на вычисление порядка элемента в прямом произведении,Время: 38:15,алгебра_семинар_3,"[Время: 36:35, Время: 59:05, Время: 80:45]",✔️ [1]
6,Упражнение: найти количество циклических образующих,Время: 47:25,алгебра_семинар_3,"[Время: 43:15, Время: 47:25, Время: 31:35]",✔️ [1]
7,Формула обратного отображения в КТО,Время: 72:25,алгебра_семинар_3,"[Время: 68:15, Время: 75:45, Время: 74:55]",
8,Решение системы сравнений по алгоритму Евклида,Время: 69:05,алгебра_семинар_3,"[Время: 70:45, Время: 69:55, Время: 71:35]",✔️ [1]
9,Проекция на подпространство через произвольный базис,Время: 70:45,линал_лекция_24,"[Время: 69:55, Время: 69:05, Время: 51:35]",✔️ [1]


Evaluate common program with accuracy@1...
Average Metric: 25.00 / 42 (59.5%): 100%|██████████| 42/42 [00:02<00:00, 14.05it/s]

2025/05/06 15:19:44 INFO dspy.evaluate.evaluate: Average Metric: 25 / 42 (59.5%)


,query,example_ranked_context,lection,pred_ranked_context,accuracy_at_1_metric
0,Определение произведения групп,Время: 44:05,алгебра_лекция_3,[],
1,Определение конечной абелевой группы,Время: 49:05,алгебра_лекция_3,[],
2,Теорема о представлении абелевых групп через произведение циклических,Время: 50:45,алгебра_лекция_3,"[Время: 49:55, Время: 55:45, Время: 76:35]",✔️ [1]
3,Китайская теорема об остатках,Время: 57:25,алгебра_лекция_3,"[Время: 59:55, Время: 57:25, Время: 74:05]",
4,Каноническое представление абелевых групп,Время: 74:55,алгебра_лекция_3,"[Время: 49:55, Время: 75:45, Время: 76:35]",
5,Упражнение на вычисление порядка элемента в прямом произведении,Время: 38:15,алгебра_семинар_3,"[Время: 36:35, Время: 59:05, Время: 80:45]",✔️ [1]
6,Упражнение: найти количество циклических образующих,Время: 47:25,алгебра_семинар_3,"[Время: 43:15, Время: 47:25, Время: 31:35]",
7,Формула обратного отображения в КТО,Время: 72:25,алгебра_семинар_3,"[Время: 68:15, Время: 75:45, Время: 74:55]",
8,Решение системы сравнений по алгоритму Евклида,Время: 69:05,алгебра_семинар_3,"[Время: 70:45, Время: 69:55, Время: 71:35]",✔️ [1]
9,Проекция на подпространство через произвольный базис,Время: 70:45,линал_лекция_24,"[Время: 69:55, Время: 69:05, Время: 51:35]",✔️ [1]


59.52

In [7]:
teleprompter = MIPROv2(metric=accuracy_at_1_metric, auto="light")

optimized_program = teleprompter.compile(
    program,
    trainset=trainset,
    valset=valset,
    max_bootstrapped_demos=3,
    max_labeled_demos=4,
    requires_permission_to_run=False,
)

2025/05/06 15:20:42 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 7
minibatch: False
num_candidates: 3
valset size: 42

2025/05/06 15:20:42 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/05/06 15:20:42 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/05/06 15:20:42 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=3 sets of demonstrations...


Bootstrapping set 1/3
Bootstrapping set 2/3
Bootstrapping set 3/3


  5%|▌         | 3/58 [00:01<00:30,  1.78it/s]
2025/05/06 15:20:44 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 2: PROPOSE INSTRUCTION CANDIDATES <==
2025/05/06 15:20:44 INFO dspy.teleprompt.mipro_optimizer_v2: We will use the few-shot examples from the previous step, a generated dataset summary, a summary of the program code, and a randomly selected prompting tip to propose instructions.
2025/05/06 15:20:44 INFO dspy.teleprompt.mipro_optimizer_v2: 
Proposing instructions...



Bootstrapped 3 full traces after 3 examples for up to 1 rounds, amounting to 3 attempts.


2025/05/06 15:22:44 INFO dspy.teleprompt.mipro_optimizer_v2: Proposed Instructions for Predictor 0:

2025/05/06 15:22:44 INFO dspy.teleprompt.mipro_optimizer_v2: 0: Перефразирование запроса на русском языке

2025/05/06 15:22:44 INFO dspy.teleprompt.mipro_optimizer_v2: 1: Given a query in Russian, provide a step-by-step reasoning to understand the underlying intent and context of the query, and generate a paraphrased version of the query limited to 10 words. Ensure that the paraphrase captures the essence of the original query to enhance retrieval accuracy.

2025/05/06 15:22:44 INFO dspy.teleprompt.mipro_optimizer_v2: 2: Transform the given mathematical query into a concise paraphrase in Russian, ensuring it does not exceed ten words. Additionally, provide a structured reasoning process that outlines the steps to understand the query's context and intent. Utilize the example queries and their paraphrases as a guide to maintain consistency and clarity in your response.

2025/05/06 15:22:

...

2025/05/06 15:26:32 INFO dspy.evaluate.evaluate: Average Metric: 30 / 42 (71.4%)
2025/05/06 15:26:32 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 71.43 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 1', 'Predictor 1: Instruction 2', 'Predictor 1: Few-Shot Set 2'].
2025/05/06 15:26:32 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [59.52, 71.43, 69.05, 71.43, 71.43]
2025/05/06 15:26:32 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 71.43
2025/05/06 15:26:32 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/05/06 15:26:32 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 6 / 7 =====



Average Metric: 30.00 / 42 (71.4%): 100%|██████████| 42/42 [00:05<00:00,  7.64it/s]

2025/05/06 15:26:38 INFO dspy.evaluate.evaluate: Average Metric: 30 / 42 (71.4%)
2025/05/06 15:26:38 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 71.43 with parameters ['Predictor 0: Instruction 0', 'Predictor 0: Few-Shot Set 0', 'Predictor 1: Instruction 2', 'Predictor 1: Few-Shot Set 2'].
2025/05/06 15:26:38 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [59.52, 71.43, 69.05, 71.43, 71.43, 71.43]
2025/05/06 15:26:38 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 71.43
2025/05/06 15:26:38 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/05/06 15:26:38 INFO dspy.teleprompt.mipro_optimizer_v2: ===== Trial 7 / 7 =====



Average Metric: 19.00 / 42 (45.2%): 100%|██████████| 42/42 [01:01<00:00,  1.46s/it]

2025/05/06 15:27:39 INFO dspy.evaluate.evaluate: Average Metric: 19 / 42 (45.2%)
2025/05/06 15:27:39 INFO dspy.teleprompt.mipro_optimizer_v2: Score: 45.24 with parameters ['Predictor 0: Instruction 2', 'Predictor 0: Few-Shot Set 1', 'Predictor 1: Instruction 0', 'Predictor 1: Few-Shot Set 1'].
2025/05/06 15:27:39 INFO dspy.teleprompt.mipro_optimizer_v2: Scores so far: [59.52, 71.43, 69.05, 71.43, 71.43, 71.43, 45.24]
2025/05/06 15:27:39 INFO dspy.teleprompt.mipro_optimizer_v2: Best score so far: 71.43
2025/05/06 15:27:39 INFO dspy.teleprompt.mipro_optimizer_v2: =======================


2025/05/06 15:27:39 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 71.43!


In [8]:
print(f"Evaluate optimized program with accuracy@3...")
evaluate_3(optimized_program, devset=valset)

print(f"Evaluate optimized program with accuracy@1...")
evaluate_1(optimized_program, devset=valset)

Evaluate optimized program with accuracy@3...
  0%|          | 0/42 [00:00<?, ?it/s]

Average Metric: 36.00 / 42 (85.7%): 100%|██████████| 42/42 [00:10<00:00,  4.11it/s]

2025/05/06 15:28:09 INFO dspy.evaluate.evaluate: Average Metric: 36 / 42 (85.7%)


,query,example_ranked_context,lection,pred_ranked_context,accuracy_at_3_metric
0,Определение произведения групп,Время: 44:05,алгебра_лекция_3,"[Время: 44:05, Время: 47:25, Время: 48:15]",✔️ [1]
1,Определение конечной абелевой группы,Время: 49:05,алгебра_лекция_3,"[Время: 49:05, Время: 49:55, Время: 55:45]",✔️ [1]
2,Теорема о представлении абелевых групп через произведение циклических,Время: 50:45,алгебра_лекция_3,"[Время: 49:55, Время: 55:45, Время: 75:45]",✔️ [1]
3,Китайская теорема об остатках,Время: 57:25,алгебра_лекция_3,"[Время: 57:25, Время: 59:55, Время: 59:05]",✔️ [1]
4,Каноническое представление абелевых групп,Время: 74:55,алгебра_лекция_3,"[Время: 75:45, Время: 76:35, Время: 49:55]",✔️ [1]
5,Упражнение на вычисление порядка элемента в прямом произведении,Время: 38:15,алгебра_семинар_3,"[Время: 36:35, Время: 59:05, Время: 39:55]",✔️ [1]
6,Упражнение: найти количество циклических образующих,Время: 47:25,алгебра_семинар_3,"[Время: 43:15, Время: 47:25, Время: 31:35]",✔️ [1]
7,Формула обратного отображения в КТО,Время: 72:25,алгебра_семинар_3,"[Время: 68:15, Время: 75:45, Время: 15:45]",
8,Решение системы сравнений по алгоритму Евклида,Время: 69:05,алгебра_семинар_3,"[Время: 69:55, Время: 70:45, Время: 71:35]",✔️ [1]
9,Проекция на подпространство через произвольный базис,Время: 70:45,линал_лекция_24,"[Время: 69:55, Время: 69:05, Время: 68:15]",✔️ [1]


Evaluate optimized program with accuracy@1...
Average Metric: 30.00 / 42 (71.4%): 100%|██████████| 42/42 [00:06<00:00,  6.24it/s]

2025/05/06 15:28:15 INFO dspy.evaluate.evaluate: Average Metric: 30 / 42 (71.4%)


,query,example_ranked_context,lection,pred_ranked_context,accuracy_at_1_metric
0,Определение произведения групп,Время: 44:05,алгебра_лекция_3,"[Время: 44:05, Время: 47:25, Время: 48:15]",✔️ [1]
1,Определение конечной абелевой группы,Время: 49:05,алгебра_лекция_3,"[Время: 49:05, Время: 49:55, Время: 55:45]",✔️ [1]
2,Теорема о представлении абелевых групп через произведение циклических,Время: 50:45,алгебра_лекция_3,"[Время: 49:55, Время: 55:45, Время: 75:45]",✔️ [1]
3,Китайская теорема об остатках,Время: 57:25,алгебра_лекция_3,"[Время: 57:25, Время: 59:55, Время: 59:05]",✔️ [1]
4,Каноническое представление абелевых групп,Время: 74:55,алгебра_лекция_3,"[Время: 75:45, Время: 76:35, Время: 49:55]",✔️ [1]
5,Упражнение на вычисление порядка элемента в прямом произведении,Время: 38:15,алгебра_семинар_3,"[Время: 36:35, Время: 59:05, Время: 39:55]",✔️ [1]
6,Упражнение: найти количество циклических образующих,Время: 47:25,алгебра_семинар_3,"[Время: 43:15, Время: 47:25, Время: 31:35]",
7,Формула обратного отображения в КТО,Время: 72:25,алгебра_семинар_3,"[Время: 68:15, Время: 75:45, Время: 22:25]",
8,Решение системы сравнений по алгоритму Евклида,Время: 69:05,алгебра_семинар_3,"[Время: 69:55, Время: 70:45, Время: 71:35]",✔️ [1]
9,Проекция на подпространство через произвольный базис,Время: 70:45,линал_лекция_24,"[Время: 69:55, Время: 69:05, Время: 68:15]",✔️ [1]


71.43

In [9]:
optimized_program.save(f"mipro_optimized_v8_paraphrase_error_acc@1.json")